# Studying network construction: Effects of number of edges

The main objective is to check how the network cardinality measures (basically number of edges and degree of each node) is effecting the ratings.

Two examples are following.

 - A network construction with a maximum of edges.
 - A random network construction by edge probability.

### Import statements and Random fix

In [ ]:
import random
import numpy as np

random.seed(321)
np.random.seed(1234)

In [ ]:
import networkx as nx
from dfg_rating.model import factory

from dfg_rating.model.network.base_network import BaseNetwork
from dfg_rating.model.network.simple_network import RoundRobinNetwork

from dfg_rating.model.rating.ranking_rating import LeagueRating
from dfg_rating.model.rating.elo_rating import ELORating

import dfg_rating.viz.jupyter_widgets as DFGWidgets

## Creating Networks

#### standard_n := Standard 10-Round-Robin league

In [ ]:
standard_n: BaseNetwork = factory.new_network(
    'round-robin',
    teams=10,
    days_between_rounds=3,
)
standard_n.create_data()
standard_n.play()

#### max_n := Network with a maximum number of edges

In [ ]:
class EdgeCappedNetwork(RoundRobinNetwork):
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.max_edges = kwargs.get("max_edges", (self.n_teams * (self.n_teams - 1)))
    
    def fill_graph(self):
        if self.data is None:
            graph = nx.MultiDiGraph()
        else:
            graph = self.data
            
        current_edges = 0
        n_list = [n for n in range(self.n_teams)]
        graph.add_nodes_from([n for n in range(self.n_teams)])
        
        while current_edges < self.max_edges:
            u = random.choice(n_list)
            v = random.choice(n_list)
            print(u,v)
            if u==v or graph.has_edge(u,v):
                continue
            else:
                n_round = max(graph.degree[v], graph.degree[u])
                graph.add_edge(u,v, season=0, round=n_round, day=self.days_between_rounds * n_round)
                current_edges += 1
                
        self.data = graph          
        

In [ ]:
max_n = EdgeCappedNetwork(
    teams=10,
    days_between_rounds=3,   
    max_edges=5
)
max_n.create_data()
max_n.play()

#### random_n := Network with edge probability

In [ ]:
class RandomNetwork(RoundRobinNetwork):
    """
    Choses each of the possible [n(n-1)]/2 edges with probability p.
    """
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.edge_probability = kwargs.get("edge_probability", 1)
    
    def fill_graph(self):
        if self.data is None:
            graph = nx.MultiDiGraph()
        else:
            graph = self.data
            
        current_edges = 0
        n_list = [n for n in range(self.n_teams)]
        graph.add_nodes_from([n for n in range(self.n_teams)])
        
        for u in range(self.n_teams):
            for v in range(u+1, self.n_teams):
                if random.random() < self.edge_probability:
                    n_round = max(graph.degree[v], graph.degree[u])
                    graph.add_edge(u,v, season=0, round=n_round, day=self.days_between_rounds * n_round)
        self.data = graph 

In [ ]:
random_n = RandomNetwork(
    teams=10,
    days_between_rounds=3,   
    edge_probability=0.5
)
random_n.create_data()
random_n.play()

## Explore networks

In [ ]:
network_explorer = DFGWidgets.NetworkExplorer(network=random_n)

In [ ]:
network_explorer.run('inline')

### Adding Ranking rating and ELO Rating to the networks

In [ ]:
league_ranking = LeagueRating()
elo = ELORating(trained=True)

In [ ]:
for network_n in [standard_n, max_n, random_n]:
    network_n.add_rating(league_ranking, 'league_ranking')
    network_n.add_rating(elo, 'ELO')

In [ ]:
ratings_explorer = DFGWidgets.RatingsExplorer(network=random_n)

In [ ]:
ratings_explorer.run('inline', port=8051)